In [7]:
import pystan as st
import pickle

# Model 8.2 in Stan

In [9]:
model_code = """
data {
    int<lower=0> N;
    int y[N];
}
parameters {
    real<lower=0,upper=1> theta;
}
model {
    theta ~ beta(1,1) ;
    y ~ bernoulli(theta) ;
}
"""
model = st.StanModel(model_code=model_code)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_2b1fc2ba37ac1d59cb69bdd0bcab7452 NOW.


In [14]:
N = 50
z = 10
data = dict(N=N, y=[1]*z + [0]*(N-z))
fit = model.sampling(data=data, iter=1000, warmup=200, chains=3)

In [24]:
fit.extract()['lp__']

array([-26.86505305, -27.21142458, -26.8617048 , ..., -26.86612623,
       -26.83618903, -28.39039841])

In [37]:
fit.extract(permuted=False, inc_warmup=False).shape

/Users/lbrinkm/projects/doing_bayesian_da/.venv/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: The 'warn' function is deprecated, use 'warning' instead
  """Entry point for launching an IPython kernel.


(800, 3, 2)

In [ ]:

with open('model.pkl', 'wb') as f:
    pickle.dump(model, f, protocol=pickle.HIGHEST_PROTOCOL)

# load it at some future point
with open('model.pkl', 'rb') as f:
    model = pickle.load(f)

# run with different data
fit = model.sampling(data=dict(N=5, y=[1, 1, 0, 1, 0]))